#TASK:
##+ Xây dựng cơ chế hỏi ngược (Clarification) để xử lý các câu hỏi mơ hồ. khi người dùng hỏi chung chung (ví dụ: học phí), hệ thống sẽ liệt kê các trường hợp liên quan (học phí quốc phòng, học phí tiếng Anh, phí tín chỉ, …) và hỏi lại người dùng muốn hỏi cụ thể phần nào (Cái này dễ làm cho nó lưu lại lịch sử trước đó để có biết và trả lời lại là đc, ko làm cũng đc nhưng mà làm thì dễ debate)
##+Làm đánh giá hai phần retrieval vs generative (Dùng một model to hơn để đánh giá với cơ chế retrieval, có thể làm tương tự với generative (Hoặc đánh giá bằng tay kiểu hỏi 20-30 câu từ dễ đến khó để xem nó trả lời đc bao nhiêu câu))

#Import thư viện (Bắt buộc)

In [ ]:
!pip install -q -U torch transformers sentence-transformers bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.

In [ ]:
!pip install -q rank_bm25

In [ ]:
!pip install -q -U torch transformers sentence-transformers bitsandbytes accelerate rank_bm25 streamlit pyngrok
!npm install localtunnel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 29.7 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
added 22 packages in 5s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋npm notice
npm notice New major version of npm available! 10.8.2 -> 11.7.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.7.0
npm notice To update run: npm install -g npm@11.7.0
npm notice
⠋

#Tải model sinh
Con AI này khá to (7B), nên nén nó lại cho nhẹ để chạy được trên Google Colab mà không bị lỗi thiếu bộ nhớ.
Không có train gì hết, cũng không cần chỉnh gì phức tạp.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "Qwen/Qwen2.5-7B-Instruct"

# Cấu hình nén 4-bit để chạy mượt trên Colab T4
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

print(f"Đang tải LLM {model_id}...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
print("LLM đã sẵn sàng!")

Đang tải LLM Qwen/Qwen2.5-7B-Instruct...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

LLM đã sẵn sàng!


#Code test (Nếu chạy web thì ko cần chạy mấy cell này)

Cell này đọc dữ liệu đã chia sẵn (chunk) từ file chunking_file.json vào bộ nhớ, để mấy bước phía sau dùng lại (như embedding, RAG, tìm kiếm…)

In [ ]:
import json

# Thay vì gõ tay, ta đọc thẳng từ file
with open('chunking_file.json', 'r', encoding='utf-8') as f:
    raw_data = json.load(f)

print(f"✅ Đã load thành công {len(raw_data)} chunks từ file!")
# Kiểm tra thử 1 dòng xem đúng không
print("Ví dụ:", raw_data[0][:100], "...")

✅ Đã load thành công 125 chunks từ file!
Ví dụ: SỔ TAY SINH VIÊN > Lời nói đầu: Các bạn sinh viên thân mến! Để hoàn thành tốt việc học tập tại Trườn ...


Cell này chuẩn bị 2 công cụ tìm kiếm tài liệu để dùng cho RAG:

BM25: tìm tài liệu theo từ khóa (kiểu gõ từ nào thì tìm đoạn có từ đó).

Reranker: dùng AI chấm điểm lại các kết quả tìm được, để chọn ra đoạn liên quan nhất với câu hỏi

In [ ]:
from rank_bm25 import BM25Okapi
from sentence_transformers import CrossEncoder
import numpy as np

# 1. Chuẩn bị cho Keyword Search (BM25)
# Tách từ đơn giản bằng khoảng trắng (đủ dùng cho demo)
tokenized_corpus = [doc.lower().split(" ") for doc in raw_data]
bm25 = BM25Okapi(tokenized_corpus)

# 2. Chuẩn bị Reranker (Mô hình chấm điểm lại)
print("⏳ Đang tải model Reranker (BAAI/bge-reranker-v2-m3)...")
reranker = CrossEncoder('BAAI/bge-reranker-v2-m3', max_length=512, device='cuda')

print("✅ Đã khởi tạo xong BM25 và Reranker!")

⏳ Đang tải model Reranker (BAAI/bge-reranker-v2-m3)...


config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

✅ Đã khởi tạo xong BM25 và Reranker!


#Đoạn code này dùng để tạo mô hình nhúng (embedding), biến văn bản thành vector
Cell này chuẩn bị hệ thống tìm tài liệu thông minh cho RAG và tạo ra hàm hybrid_search() để dùng sau này.

Nó làm mấy việc chính:

Biến toàn bộ dữ liệu text thành vector để tìm theo ý nghĩa

Chuẩn bị tìm kiếm theo từ khóa (BM25)

Load AI chấm điểm lại (reranker) để chọn đoạn liên quan nhất

Kết hợp cả vector search + keyword search, rồi chấm lại lần cuối

Sau khi chạy xong:

Dữ liệu đã sẵn sàng để tìm kiếm

Có thể gọi hybrid_search("câu hỏi")

Hàm sẽ trả về Top K đoạn text liên quan nhất để đưa cho AI trả lời

In [ ]:
from sentence_transformers import SentenceTransformer, CrossEncoder
from rank_bm25 import BM25Okapi
import numpy as np
import json

# 1. Load Model Embedding (E5)
print("Đang tải model Embedding (E5-Large)...")
embed_model = SentenceTransformer('intfloat/multilingual-e5-large', device='cuda')

# 2. Vector hóa dữ liệu
print("Đang vector hóa dữ liệu...")
# Lưu ý: raw_data phải được load từ trước đó (bước đọc file json)
formatted_data = ["passage: " + chunk for chunk in raw_data]
corpus_embeddings = embed_model.encode(formatted_data, normalize_embeddings=True)

print("Đang khởi tạo BM25...")
tokenized_corpus = [doc.lower().split(" ") for doc in raw_data]
bm25 = BM25Okapi(tokenized_corpus)

print("Đang tải Reranker...")
reranker = CrossEncoder('BAAI/bge-reranker-v2-m3', max_length=512, device='cuda')


# 5. Định nghĩa hàm Hybrid Search (Đoạn code của bạn nằm ở đây)
def hybrid_search(query, top_k=3):
    # --- BƯỚC A: TÌM KIẾM ỨNG VIÊN (RETRIEVAL) ---

    # 1. Vector Search
    query_vec = embed_model.encode(["query: " + query], normalize_embeddings=True)
    sim_scores = np.dot(corpus_embeddings, query_vec.T).flatten()
    vector_top_ind = np.argsort(sim_scores)[-10:][::-1]

    # 2. Keyword Search (BM25)
    tokenized_query = query.lower().split(" ")
    bm25_scores = bm25.get_scores(tokenized_query)
    bm25_top_ind = np.argsort(bm25_scores)[-10:][::-1]

    # 3. Gộp kết quả
    combined_indices = list(set(list(vector_top_ind) + list(bm25_top_ind)))
    candidate_chunks = [raw_data[i] for i in combined_indices]

    if not candidate_chunks:
        return []

    # --- BƯỚC B: CHẤM ĐIỂM LẠI (RERANKING) ---
    rerank_inputs = [[query, chunk] for chunk in candidate_chunks]
    scores = reranker.predict(rerank_inputs)

    # Sắp xếp và lấy Top K
    ranked_results = sorted(zip(candidate_chunks, scores), key=lambda x: x[1], reverse=True)
    final_results = [res[0] for res in ranked_results[:top_k]]

    return final_results

Đang tải model Embedding (E5-Large)...


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

Đang vector hóa dữ liệu...
Đang khởi tạo BM25...
Đang tải Reranker...


#Đoạn này test code thẳng trên colab, ghi câu hỏi ở hàm q và chạy để trả lời

In [ ]:
def rag_chat_hybrid(user_question):
    print(f"\n🔍 Đang tìm kiếm (Hybrid): '{user_question}'...")

    # GỌI HÀM HYBRID MỚI TẠI ĐÂY
    retrieved_chunks = hybrid_search(user_question, top_k=3)

    context_text = "\n\n".join(retrieved_chunks)

    # Prompt giữ nguyên như code cũ của bạn
    messages =  [
    {
        "role": "system",
        "content": "Bạn là **Trợ lý ảo dành cho sinh viên Trường Đại học Khoa học – Đại học Huế**.\n\n"
                   "Nhiệm vụ của bạn:\n"
                   "1. Luôn trả lời dựa trên **“Thông tin tham khảo”** mà người dùng cung cấp ở mỗi câu hỏi.\n"
                   "2. Chỉ được sử dụng thông tin trong tài liệu đó.\n"
                   "3. Nếu người dùng hỏi điều KHÔNG có trong “Thông tin tham khảo”, bạn phải trả lời:\n"
                   "   “Xin lỗi, thông tin này không nằm trong tài liệu người dùng đã cung cấp. Tôi không thể trả lời.”\n"
                   "4. Tuyệt đối không tự bịa thông tin.\n"
                   "5. Nếu có nhiều đoạn tài liệu, bạn phải:\n"
                   "   - Đọc tất cả\n"
                   "   - Tổng hợp chính xác, dễ hiểu\n"
                   "   - Giữ ngữ nghĩa đúng với tài liệu gốc\n"
                   "6. Khi trả lời:\n"
                   "   - Viết rõ ràng, mạch lạc\n"
                   "   - Giải thích đơn giản, thân thiện\n"
                   "   - Không dùng thuật ngữ khó nếu không cần thiết\n"
                   "7. Nếu câu hỏi của người dùng không liên quan đến tài liệu (ví dụ hỏi lập trình, đời sống…):\n"
                   "   → Trả lời: “Câu hỏi này nằm ngoài phạm vi Thông tin tham khảo.”\n\n"
                   "Hãy luôn tuân thủ các quy tắc trên trong mọi tình huống.\n"
    },
    {
        "role": "user",
        "content": f"Thông tin tham khảo:\n{ctx}\n\nCâu hỏi: {query}"
    }
]

    # Sinh câu trả lời (Giữ nguyên logic cũ)
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1024,
        temperature=0.3,
        do_sample=True
    )

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    final_answer = response.split("assistant\n")[-1]

    return final_answer

# --- TEST THỬ NGAY ---
# Câu hỏi này cần Keyword Search để bắt chính xác số "81"

# Câu hỏi này cần Vector Search để hiểu ngữ nghĩa "nghèo" ~ "chính sách"
q = "Học lại khi nào"
ans = rag_chat_hybrid(q)
print(f"🤖 Trả lời:\n{ans}")


🔍 Đang tìm kiếm (Hybrid): 'Học lại khi nào'...


NameError: name 'ctx' is not defined

In [ ]:
from google.colab import drive

# 1. Kết nối Google Drive
drive.mount('/content/drive')

#Deploy bằng web

Model đã đc tải sẵn: https://drive.google.com/drive/folders/151NSXergkOFiUXm0Z6Nna9jjzlGeBxri?usp=sharing thêm tối tắt vào drive là đc
#Đang thiếu database, yêu cầu đẩy file json vào chormaDB
Copy hết đoạn này rồi kêu gpt làm dùm là đc


In [ ]:
%%writefile app.py
import streamlit as st
import json
import torch
import gc
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder
from rank_bm25 import BM25Okapi
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# -----------------------------------------------------
#  🎨 TÙY CHỈNH GIAO DIỆN
# -----------------------------------------------------
st.set_page_config(page_title="RAG Hybrid Chatbot", page_icon="🎓", layout="wide")

st.markdown("""
<style>
.chat-container {
    max-width: 820px;
    margin-left: auto;
    margin-right: auto;
}

[data-testid="stChatMessageContent"] {
    font-size: 17px;
    line-height: 1.5;
}

.user-msg {
    background-color: #DCF8C6 !important;
    border-radius: 12px !important;
    padding: 10px !important;
}

.assistant-msg {
    background-color: #F1F0F0 !important;
    border-radius: 12px !important;
    padding: 10px !important;
}
</style>
""", unsafe_allow_html=True)

st.title("Trợ lý Quy chế Sinh viên")


# -----------------------------------------------------
#  📂 1. TẢI TÀI NGUYÊN (CACHE)
# -----------------------------------------------------
@st.cache_resource
def load_resources():
    status = st.sidebar.empty()
    status.info("⏳ Đang load dữ liệu...")

    # Load JSON
    with open("chunking_file.json", "r", encoding="utf-8") as f:
        raw_data = json.load(f)

    # Embedding
    status.info("⏳ Đang load Embedding (E5)...")
    embed_model = SentenceTransformer("intfloat/multilingual-e5-large", device="cuda")
    formatted = ["passage: " + c for c in raw_data]
    corpus_emb = embed_model.encode(formatted, normalize_embeddings=True, show_progress_bar=True)

    del formatted
    gc.collect()

    # BM25
    status.info("⏳ Khởi tạo BM25...")
    tokenized = [doc.lower().split(" ") for doc in raw_data]
    bm25 = BM25Okapi(tokenized)

    del tokenized
    gc.collect()

    # Reranker
    status.info("⏳ Load Reranker...")
    reranker = CrossEncoder("BAAI/bge-reranker-v2-m3", device="cuda")

    # LLM
    status.info("⏳ Đang load Qwen 7B (4-bit)...")
    bnb = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )

    tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B-Instruct")
    model = AutoModelForCausalLM.from_pretrained(
        "Qwen/Qwen2.5-7B-Instruct",
        quantization_config=bnb,
        device_map="auto"
    )

    status.success("✅ Xong!")
    return raw_data, embed_model, corpus_emb, bm25, reranker, tokenizer, model


raw_data, embed_model, corpus_embeddings, bm25, reranker, tokenizer, model = load_resources()


# -----------------------------------------------------
#  🔍 2. HYBRID SEARCH
# -----------------------------------------------------
def hybrid_search(query, top_k=3):
    query_vec = embed_model.encode(["query: " + query], normalize_embeddings=True)
    sim = np.dot(corpus_embeddings, query_vec.T).flatten()
    vec_top = np.argsort(sim)[-12:][::-1]

    tokens = query.lower().split(" ")
    bm25_score = bm25.get_scores(tokens)
    bm25_top = np.argsort(bm25_score)[-12:][::-1]

    merged = list(set(list(vec_top) + list(bm25_top)))
    candidates = [raw_data[i] for i in merged]

    rerank_input = [[query, c] for c in candidates]
    scores = reranker.predict(rerank_input)
    ranked = sorted(zip(candidates, scores), key=lambda x: x[1], reverse=True)
    return [c[0] for c in ranked[:top_k]]


# -----------------------------------------------------
#  🤖 3. SINH CÂU TRẢ LỜI
# -----------------------------------------------------
def generate_response(query):
    ctx = "\n\n".join(hybrid_search(query))

    messages =  [
    {
        "role": "system",
        "content": "Bạn là **Trợ lý ảo dành cho sinh viên Trường Đại học Khoa học – Đại học Huế**.\n\n"
                   "Nhiệm vụ của bạn:\n"
                   "1. Luôn trả lời dựa trên **“Thông tin tham khảo”** mà người dùng cung cấp ở mỗi câu hỏi.\n"
                   "2. Chỉ được sử dụng thông tin trong tài liệu đó.\n"
                   "3. Nếu người dùng hỏi điều KHÔNG có trong “Thông tin tham khảo”, bạn phải trả lời:\n"
                   "   “Xin lỗi, thông tin này không nằm trong tài liệu người dùng đã cung cấp. Tôi không thể trả lời.”\n"
                   "4. Tuyệt đối không tự bịa thông tin.\n"
                   "5. Nếu có nhiều đoạn tài liệu, bạn phải:\n"
                   "   - Đọc tất cả\n"
                   "   - Tổng hợp chính xác, dễ hiểu\n"
                   "   - Giữ ngữ nghĩa đúng với tài liệu gốc\n"
                   "6. Khi trả lời:\n"
                   "   - Viết rõ ràng, mạch lạc\n"
                   "   - Giải thích đơn giản, thân thiện\n"
                   "   - Không dùng thuật ngữ khó nếu không cần thiết\n"
                   "7. Nếu câu hỏi của người dùng không liên quan đến tài liệu (ví dụ hỏi lập trình, đời sống…):\n"
                   "   → Trả lời: “Câu hỏi này nằm ngoài phạm vi Thông tin tham khảo.”\n\n"
                   "Hãy luôn tuân thủ các quy tắc trên trong mọi tình huống.\n"
    },
    {
        "role": "user",
        "content": f"Thông tin tham khảo:\n{context_text}\n\nCâu hỏi: {query}"
    }
]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inp = tokenizer([text], return_tensors="pt").to(model.device)

    out = model.generate(**inp, max_new_tokens=600, temperature=0.3)
    answer = tokenizer.decode(out[0], skip_special_tokens=True)
    return answer.split("assistant\n")[-1]


# -----------------------------------------------------
#  💬 4. GIAO DIỆN CHAT
# -----------------------------------------------------
if "messages" not in st.session_state:
    st.session_state.messages = []

st.sidebar.subheader("⚙️ Tuỳ chọn")
if st.sidebar.button("🧹 Xóa lịch sử chat"):
    st.session_state.messages = []
    st.rerun()

# Lịch sử chat
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

# Nhập câu hỏi
if prompt := st.chat_input("Nhập câu hỏi của bạn..."):
    st.session_state.messages.append({"role": "user", "content": prompt})

    with st.chat_message("user"):
        st.markdown(prompt)

    with st.chat_message("assistant"):
        with st.spinner("Đang xử lý..."):
            answer = generate_response(prompt)
            st.markdown(answer)

    st.session_state.messages.append({"role": "assistant", "content": answer})


Overwriting app.py


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Vào grok để tạo key api, ghi vào một file txt đặt tên là token.txt rồi đẩy lên colab

In [ ]:
# Đọc token từ file txt
with open("token.txt", "r") as f:
    NGROK_TOKEN = f.read().strip()

print("🔑 Token đã được load từ file.")

from pyngrok import ngrok

# Tắt kết nối cũ nếu có
ngrok.kill()

# Set token
!ngrok config add-authtoken $NGROK_TOKEN

# Mở cổng 8501
public_url = ngrok.connect(8501).public_url
print(f"🚀 LINK MỚI CỦA BẠN: {public_url}")

# Chạy Streamlit
!streamlit run app.py


🔑 Token đã được load từ file.
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🚀 LINK MỚI CỦA BẠN: https://uncausative-dexterously-tandra.ngrok-free.dev





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.133.19:8501

2026-01-06 05:10:25.279110: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767676225.323442    5420 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767676225.337963    5420 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767676225.388010    5420 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767676225.388048    5420 computation_placer.cc:177] computation placer alre